<div class="alert alert-block alert-info text-center">
    <H1> PHASE I (cont.) - preparing our data </H1>
</div>


## Getting all dataframes ready to explore

- Data In & Data Out
- Getting the price history for each ticker
- Getting business information to enrich our analysis
- Understanding the big picture

In [1]:
import pandas as pd
from glob import glob
from time import strftime, sleep
import numpy as np
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay
import yfinance as yf
yf.pdr_override()
import BizExtractor

def clean_header(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '', regex=False).str.replace('(', \
                '', regex=False).str.replace(')', '', regex=False).str.replace(' ', '_', regex=False).str.replace('_/_', '/', regex=False)
    
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

def datetime_maker(df, datecol):
    df[datecol] = pd.to_datetime(df[datecol])

#### Import the last transactions_finaldf from Phase I

In [2]:
last_file = sorted(glob('../outputs/transactions_all/*finaldf*.xlsx'))[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
all_transactions = pd.read_excel(last_file, engine='openpyxl')
all_transactions.date = pd.to_datetime(all_transactions.date, format='%d/%m/%Y')

transactions_all\transactions_finaldf_2021-12-02_02h36m.xlsx


In [3]:
all_tickers = list(all_transactions['ticker'].unique())
print('You have {} different stocks'.format(len(all_tickers)))

You have 47 different stocks


In [4]:
all_tickers

['MMM',
 'TSLA',
 'NFLX',
 'AAPL',
 'MSFT',
 'RUN',
 'CVS',
 'CSCO',
 'DIS',
 'JNJ',
 'AMD',
 'INTC',
 'PYPL',
 'IBM',
 'FB',
 'NWS',
 'AMZN',
 'VIAC',
 'GOOGL',
 'TTD',
 'NVDA',
 'WMT',
 'PTON',
 'MRNA',
 'ZEN',
 'MO',
 'FVRR',
 'SHOP',
 'PAYS',
 'TWTR',
 'STOR',
 'ZI',
 'MCD',
 'NKLA',
 'BA',
 'JPM',
 'CVX',
 'PLUG',
 'SYF',
 'NIO',
 'MGM',
 'DKNG',
 'DOCU',
 'FDX',
 'PLTR',
 'PLL',
 'RIOT']

In [4]:
# All transactions without the delisted stocks
# final_filtered = all_transactions[~all_transactions.ticker.isin(blacklist)]

### Collecting the price history for all tickers

- You can define the start date for the history below
- Datareader will get each stock individually
- all_data will have all the prices for every ticker

In [5]:
ly = datetime.today().year-1
today = datetime.today()
start_sp = datetime(2020, 1, 1)
end_sp = today
start_stocks = datetime(2020, 1, 1)
end_stocks = today
start_ytd = datetime(ly, 12, 31) + BDay(1) # to get the first business day

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['ticker', 'date']))
               
all_data = get(all_tickers, start_stocks, end_stocks)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
all_data

Open        High         Low       Close   Adj Close  \
ticker date                                                                     
MMM    2020-01-02  177.679993  180.009995  177.139999  180.000000  166.588974   
       2020-01-03  177.020004  178.660004  175.630005  178.449997  165.154465   
       2020-01-06  177.149994  178.710007  176.350006  178.619995  165.311783   
       2020-01-07  178.279999  178.509995  176.820007  177.899994  164.645447   
       2020-01-08  178.000000  181.500000  177.649994  180.630005  167.172043   
...                       ...         ...         ...         ...         ...   
RIOT   2022-02-18   17.680000   17.840000   16.730000   16.969999   16.969999   
       2022-02-22   16.114000   16.996000   15.710000   15.950000   15.950000   
       2022-02-23   16.580000   16.790001   15.020000   15.070000   15.070000   
       2022-02-24   13.690000   16.389999   13.630000   16.250000   16.250000   
       2022-02-25   16.410000   16.700001   15.585000   16.219999   16.219999   

                     Volume  
ticker date                  
MMM    2020-01-02   3601700  
       2020-01-03   2466900  
       2020-01-06   1998000  
       2020-01-07   2173000  
       2020-01-08   2758300  
...                     ...  
RIOT   2022-02-18   6555100  
       2022-02-22   7301500  
       2022-02-23   7533100  
       2022-02-24  12252600  
       2022-02-25   7543600  

[25227 rows x 6 columns]

In [7]:
clean_header(all_data)

### Saving the price history

<div class="alert alert-block alert-danger">
<b>Please Note:</b><br>If an item does not have price history, we have to remove it from the portfolio.
    <br>This may happen if the stock gets delisted.
</div>

In [8]:
# We catch the tickers in a list
blacklist = []
for tick in all_tickers:
    try:
        all_data.loc[tick].to_csv('../outputs/price_hist/{}_price_hist.csv'.format(tick))
    except KeyError:
        blacklist.append(tick)
        print(f'Ticker {tick} has no price history to save.')
        pass
blacklist

[]

In [9]:
filt_tickers = [tick for tick in all_tickers if tick not in blacklist]

In [10]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 25227 entries, ('MMM', Timestamp('2020-01-02 00:00:00')) to ('RIOT', Timestamp('2022-02-25 00:00:00'))
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       25227 non-null  float64
 1   high       25227 non-null  float64
 2   low        25227 non-null  float64
 3   close      25227 non-null  float64
 4   adj_close  25227 non-null  float64
 5   volume     25227 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 1.2+ MB


<div class="alert alert-block alert-warning">
<b>MEGA datasets:</b><br>These will be handy later when we need to quickly access specific stocks.
    <br>We use the all_data dataframe, along with the transactions, to create a day-by-day view of each stock value.
</div>

#### MEGA_DICT is dictionary with all the tickers as keys, and their ticker prices as DF
#### MEGA_DF is a dataframe with all the DF's from MEGA_DICT, concatenated along the columns. Can use filter to select columns

In [11]:
MEGA_DICT = {}  # you have to create it first
min_date = '2020-01-01'  # optional
TX_COLUMNS = ['date','ticker', 'cashflow', 'cml_units', 'cml_cost', 'gain_loss']
tx_filt = all_transactions[TX_COLUMNS]  # keeping just the most relevant ones for now

for ticker in filt_tickers:
    prices_df = all_data[all_data.index.get_level_values('ticker').isin([ticker])].reset_index()
    ## Can add more columns like volume!
    PX_COLS = ['date', 'adj_close']
    prices_df = prices_df[prices_df.date >= min_date][PX_COLS].set_index(['date'])
    # Making sure we get sameday transactions
    tx_df = tx_filt[tx_filt.ticker==ticker].groupby('date').agg({'cashflow': 'sum',
                                                                 'cml_units': 'last',
                                                                 'cml_cost': 'last',
                                                                 'gain_loss': 'sum'})
    # Merging price history and transactions dataframe
    tx_and_prices = pd.merge(prices_df, tx_df, how='outer', left_index=True, right_index=True).fillna('-')
    # This is to fill the days that were not in our transaction dataframe
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace='-', method='ffill')
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace='-', method='ffill')
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace='-', method='ffill')
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace='-', value=0)
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace='-', value=0)
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace='-', value=0)
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].replace(to_replace='-', value=0)
    # Cumulative sum for the cashflow
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].cumsum()
    tx_and_prices[['cml_cost', 'cml_units']] = tx_and_prices[['cml_cost', 'cml_units']].apply(pd.to_numeric)
    tx_and_prices['avg_price'] = round(tx_and_prices['cml_cost']/tx_and_prices['cml_units'],3)
    tx_and_prices['mktvalue'] = round(tx_and_prices['cml_units']*tx_and_prices['adj_close'],3)
    tx_and_prices = tx_and_prices.add_prefix(ticker+'_')
    # Once we're happy with the dataframe, add it to the dictionary
    MEGA_DICT[ticker] = tx_and_prices

In [12]:
MEGA_DICT['AAPL'].head(10)

,AAPL_adj_close,AAPL_cashflow,AAPL_cml_units,AAPL_cml_cost,AAPL_gain_loss,AAPL_avg_price,AAPL_mktvalue
date,,,,,,,
2020-01-02,73.894333,0.0,0.0,0.0,0.0,NaN,0.000
2020-01-03,73.175926,0.0,0.0,0.0,0.0,NaN,0.000
2020-01-06,73.759010,0.0,0.0,0.0,0.0,NaN,0.000
2020-01-07,73.412125,0.0,0.0,0.0,0.0,NaN,0.000
2020-01-08,74.593033,0.0,0.0,0.0,0.0,NaN,0.000
2020-01-09,76.177460,-1854.0,24.0,1854.0,0.0,77.25,1828.259
2020-01-10,76.349670,-1854.0,24.0,1854.0,0.0,77.25,1832.392
2020-01-13,77.980835,-1854.0,24.0,1854.0,0.0,77.25,1871.540
2020-01-14,76.927834,-1854.0,24.0,1854.0,0.0,77.25,1846.268


In [13]:
MEGA_DF = pd.concat(MEGA_DICT.values(), axis=1)
MEGA_DF.to_csv('../outputs/mega/MEGA_DF_{}.csv'.format(get_now()))
MEGA_DF.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 543 entries, 2020-01-02 to 2022-02-25
Columns: 329 entries, MMM_adj_close to RIOT_mktvalue
dtypes: float64(329)
memory usage: 1.4 MB


In [14]:
last_file = sorted(glob('../outputs/mega/MEGA*.csv'))[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
MEGA_DF = pd.read_csv(last_file)

MEGA_DF['date'] = pd.to_datetime(MEGA_DF['date'])
MEGA_DF.set_index('date', inplace=True)

mega\MEGA_DF_2022-02-27_00h28m.csv


## Creating the daily snapshots of our portfolio

- Taking the MEGA_DF dataframe, we keep just the "market value" column for each stock
- If we sum the rows, it will show us how much our portfolio was worth on that day
- Adding SP500 for reference and calculating some metrics
- saving the portf_allvalues dataframe as csv

In [15]:
portf_allvalues = MEGA_DF.filter(regex='mktvalue').fillna(0)
portf_allvalues['portf_value'] = portf_allvalues.sum(axis=1)
portf_allvalues['portf_value']

date
2020-01-02        0.000
2020-01-03        0.000
2020-01-06        0.000
2020-01-07        0.000
2020-01-08     3351.420
                ...    
2022-02-18    81508.880
2022-02-22    80075.964
2022-02-23    78071.995
2022-02-24    81040.433
2022-02-25    82586.709
Name: portf_value, Length: 543, dtype: float64

In [16]:
# For the S&P500 price return
# You can use other symbols. Look it up on yahoo finance
sp500 = pdr.get_data_yahoo('^GSPC', start_stocks, end_sp)
clean_header(sp500)

[*********************100%***********************]  1 of 1 completed


In [17]:
#getting the pct change
portf_allvalues = portf_allvalues.join(sp500['adj_close'], how='inner')
portf_allvalues.rename(columns={'adj_close': 'sp500_mktvalue'}, inplace=True)
portf_allvalues['ptf_value_pctch'] = (portf_allvalues['portf_value'].pct_change()*100).round(2)
portf_allvalues['sp500_pctch'] = (portf_allvalues['sp500_mktvalue'].pct_change()*100).round(2)
portf_allvalues['ptf_value_diff'] = (portf_allvalues['portf_value'].diff()).round(2)
portf_allvalues['sp500_diff'] = (portf_allvalues['sp500_mktvalue'].diff()).round(2)
portf_allvalues.index.name = 'date'
portf_allvalues.reset_index(inplace=True)
portf_allvalues

,date,MMM_mktvalue,TSLA_mktvalue,NFLX_mktvalue,AAPL_mktvalue,MSFT_mktvalue,RUN_mktvalue,CVS_mktvalue,CSCO_mktvalue,DIS_mktvalue,...,FDX_mktvalue,PLTR_mktvalue,PLL_mktvalue,RIOT_mktvalue,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff
0,2020-01-02,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.000,...,0.000,0.00,0.0,0.00,0.000,3257.850098,NaN,NaN,NaN,NaN
1,2020-01-03,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.000,...,0.000,0.00,0.0,0.00,0.000,3234.850098,NaN,-0.71,0.00,-23.00
2,2020-01-06,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.000,...,0.000,0.00,0.0,0.00,0.000,3246.280029,NaN,0.35,0.00,11.43
3,2020-01-07,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.000,...,0.000,0.00,0.0,0.00,0.000,3237.179932,NaN,-0.28,0.00,-9.10
4,2020-01-08,2006.065,124.019,1221.336,0.00,0.00,0.00,0.00,0.00,0.000,...,0.000,0.00,0.0,0.00,3351.420,3253.050049,inf,0.49,3351.42,15.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,2022-02-18,1771.440,7250.051,1173.870,6022.80,3455.16,776.16,1225.56,686.52,1671.014,...,1600.056,352.64,1460.7,933.35,81508.880,4348.870117,-1.76,-0.72,-1456.58,-31.39
539,2022-02-22,1761.000,6950.144,1132.140,5915.52,3452.64,752.40,1223.04,675.60,1634.914,...,1568.088,335.36,1404.6,877.25,80075.964,4304.759766,-1.76,-1.01,-1432.92,-44.11
540,2022-02-23,1732.560,6463.778,1102.380,5762.52,3363.24,719.64,1226.88,653.28,1610.405,...,1546.848,333.76,1381.2,828.85,78071.995,4225.500000,-2.50,-1.84,-2003.97,-79.26
541,2022-02-24,1724.400,6774.514,1170.090,5858.64,3535.08,877.68,1209.24,656.76,1649.376,...,1554.624,378.56,1485.9,893.75,81040.433,4288.700195,3.80,1.50,2968.44,63.20


In [29]:
portf_allvalues.to_csv('../outputs/portfolio_df/portfolio_df_{}.csv'.format(get_now()), index=False)

## Getting Sector and Industry
### The "give me everything" method

- In order for us to get a broader view on our portfolio, we want to add the sector and industry to the dataframe
- If you want a complete view of each company, go ahead and use the GET_BIZ_DATA_ALL function
- It takes a bit longer if you have a lot of tickers on your list, but it gives you a lot of information to explore later

In [18]:
%%time
screener_all = BizExtractor.get_biz_data_all(all_tickers)

There are 3 pages
####
Overview page scrape in progress...
#### took care of page 1 out of 3
#### took care of page 2 out of 3
#### took care of page 3 out of 3
####
Performance page scrape in progress...
#### took care of page 1 out of 3
#### took care of page 2 out of 3
#### took care of page 3 out of 3
####
Technical page scrape in progress...
#### took care of page 1 out of 3
#### took care of page 2 out of 3
#### took care of page 3 out of 3
####
Ownership page scrape in progress...
#### took care of page 1 out of 3
#### took care of page 2 out of 3
#### took care of page 3 out of 3
####
Valuation page scrape in progress...
#### took care of page 1 out of 3
#### took care of page 2 out of 3
#### took care of page 3 out of 3
####
Financial page scrape in progress...
#### took care of page 1 out of 3
#### took care of page 2 out of 3
#### took care of page 3 out of 3
####
Merging everything together...
ALL DONE!
Wall time: 40.5 s


C:\FN\06 - PYTHON\202111 Dashboard Course\jupyter notebooks\BizExtractor.py:113: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Price_x', 'Change_x', 'Volume_x', 'No._x'} in the result is deprecated and will raise a MergeError in a future version.
  finviz_merged_raw = pd.merge(finviz_merged_raw,finviz_owner_df, on='Ticker')
C:\FN\06 - PYTHON\202111 Dashboard Course\jupyter notebooks\BizExtractor.py:115: FutureWarning: Passing 'suffixes' which cause duplicate columns {'No._x', 'Change_x', 'Market Cap_x', 'Price_x', 'Volume_x'} in the result is deprecated and will raise a MergeError in a future version.
  finviz_merged_raw = pd.merge(finviz_merged_raw,finviz_finance_df, on='Ticker')


In [19]:
# Visit https://finviz.com/screener.ashx

screener_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 46
Data columns (total 62 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Ticker         47 non-null     object
 1   Company        47 non-null     object
 2   Sector         47 non-null     object
 3   Industry       47 non-null     object
 4   Country        47 non-null     object
 5   Market Cap     47 non-null     object
 6   P/E            47 non-null     object
 7   Price          47 non-null     object
 8   Change         47 non-null     object
 9   Volume         47 non-null     object
 10  Perf Week      47 non-null     object
 11  Perf Month     47 non-null     object
 12  Perf Quart     47 non-null     object
 13  Perf Half      47 non-null     object
 14  Perf Year      47 non-null     object
 15  Perf YTD       47 non-null     object
 16  Volatility W   47 non-null     object
 17  Volatility M   47 non-null     object
 18  Recom          47 non-null     o

In [32]:
screener_all.to_csv('../outputs/ticker_information/ticker_information.csv', index=False)

In [20]:
cols_clean_all = ['Ticker', 'Company', 'Sector', 'Industry', 'P/E', 'Perf Week', 'Perf Month', 'Perf Quart',
             'Perf Half', 'Perf Year', 'Perf YTD', 'Volatility W', 'Volatility M', 'Recom', 'ATR',
             'SMA20', 'SMA50', 'SMA200', '52W High', '52W Low', 'RSI', 'Insider Own', 'Insider Trans',
             'Inst Own', 'Inst Trans', 'Float Short', 'Short Ratio', 'Dividend', 'LTDebt/Eq', 'Debt/Eq']
screener_all_clean = screener_all[cols_clean_all]
clean_header(screener_all_clean)
screener_all_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 46
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ticker         47 non-null     object
 1   company        47 non-null     object
 2   sector         47 non-null     object
 3   industry       47 non-null     object
 4   p/e            47 non-null     object
 5   perf_week      47 non-null     object
 6   perf_month     47 non-null     object
 7   perf_quart     47 non-null     object
 8   perf_half      47 non-null     object
 9   perf_year      47 non-null     object
 10  perf_ytd       47 non-null     object
 11  volatility_w   47 non-null     object
 12  volatility_m   47 non-null     object
 13  recom          47 non-null     object
 14  atr            47 non-null     object
 15  sma20          47 non-null     object
 16  sma50          47 non-null     object
 17  sma200         47 non-null     object
 18  52w_high       47 non-null     o

### The faster method
<div class="alert alert-block alert-info">
<b>Tip:</b> <br>If you just want to get information about industry and sector, you can use the faster method which scrapes just one page.<br>You need to specify which one, so make sure to check the function documentation!<br>
</div>

- *Don't forget to adapt the variable name from screener_all_clean to finviz_merged_clean on the Final Last Positions first cell*

In [21]:
%%time
finviz_merged_raw = BizExtractor.get_finviz_screener('OVERVIEW', all_tickers)

#### took care of page 1 out of 3
#### took care of page 2 out of 3
#### took care of page 3 out of 3
Wall time: 6.66 s


In [22]:
cols_clean = ['Ticker', 'Company', 'Sector', 'Industry', 'P/E']
finviz_merged_clean = finviz_merged_raw[cols_clean]
clean_header(finviz_merged_clean)
finviz_merged_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 1 to 49
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ticker    47 non-null     object
 1   company   47 non-null     object
 2   sector    47 non-null     object
 3   industry  47 non-null     object
 4   p/e       47 non-null     object
dtypes: object(5)
memory usage: 2.2+ KB


<div class="alert alert-block alert-info">
<b>Tip:</b> <br>You can use this function whenever you want as long as you have a list of stocks!<br>
</div>

In [38]:
# Quick example of an ad-hoc analysis
div_tickers = ['MAIN', 'PFLT', 'GAIN', 'AGNC', 'PBA', 'ABR', 'BDN', 'O', 'EPR', 'BGS', 'SPG', 'SLG', 'STAG']
BizExtractor.get_finviz_screener('VALUATION', div_tickers)

#### took care of page 1 out of 1


,No.,Ticker,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,EPS this Y,EPS next Y,EPS past 5Y,EPS next 5Y,Sales past 5Y,Price,Change,Volume
1,1,ABR,2.53B,8.12,9.15,1.02,3.34,1.51,6.64,-,-4.00%,-5.32%,6.50%,8.00%,33.50%,17.42,-1.41%,"1,421,466"
2,2,AGNC,8.12B,5.73,6.11,1.88,4.62,0.88,8.27,11.10,-156.60%,-14.79%,-26.40%,3.04%,-6.00%,15.33,-0.84%,"5,583,206"
3,3,BDN,2.27B,86.67,-,17.33,4.65,1.32,53.43,50.35,823.70%,11.10%,59.70%,5.00%,-2.40%,13.26,-0.08%,"1,435,086"
4,4,BGS,1.97B,23.21,14.60,-,0.99,2.35,72.51,-,72.80%,7.17%,11.00%,-0.50%,15.30%,30.13,-0.50%,"651,459"
5,5,EPR,3.42B,342.60,27.04,48.94,7.28,1.28,23.70,16.55,-220.70%,123.42%,-22.00%,7.00%,-0.30%,44.88,-1.88%,"629,338"
6,6,GAIN,546.55M,12.86,18.70,4.29,7.74,1.43,-,-,16.70%,39.70%,3.18%,3.00%,-,16.46,-1.02%,"164,175"
7,7,MAIN,3.09B,9.52,16.98,1.19,11.48,1.80,51.91,-,-78.30%,0.78%,-26.80%,8.00%,6.20%,43.97,-1.10%,"315,261"
8,8,O,38.20B,53.16,38.78,9.75,21.03,1.98,73.89,173.02,-17.10%,20.08%,1.10%,5.45%,10.00%,67.25,-0.40%,"3,938,795"
9,9,PBA,16.14B,-,15.28,-,2.22,1.34,-,-,24.80%,12.30%,0.53%,24.50%,-,29.34,-1.44%,"875,942"
10,10,PFLT,501.32M,7.44,11.54,-,6.05,1.02,-,-,-6.30%,6.70%,0.66%,-,-,12.93,-0.69%,"195,490"


# Last Positions

## Getting the latest values

- In order to get the latest position value, we need to get the latest prices from yahoo finance
- Since we also want sector and industry to be able to segment our portfolio, we will get data from finviz too


In [39]:
last_positions = all_transactions.groupby(['ticker']).agg({'cml_units': 'last', 'cml_cost': 'last',
                                                'gain_loss': 'sum', 'cashflow': 'sum'}).reset_index()
last_positions.head(10)

,ticker,cml_units,cml_cost,gain_loss,cashflow
0,AAPL,36.00,2735.88,0.0,-2735.88
1,AMD,24.00,1296.00,0.0,-1296.00
2,AMZN,1.70,4360.11,0.0,-4360.11
3,BA,7.80,1644.36,0.0,-1644.36
4,CSCO,12.00,469.44,0.0,-469.44
5,CVS,12.00,656.28,0.0,-656.28
6,CVX,18.00,1607.32,0.0,-1607.32
7,DIS,11.04,1180.84,0.0,-1180.84
8,DKNG,27.50,1080.30,0.0,-1080.30
9,DOCU,12.00,2941.56,0.0,-2941.56


<div class="alert alert-block alert-info">
<b>Tip:</b> <br> yfinance is a powerful tool too, so I'm going to leave you another example.<br>
    Make sure to try it out!
</div>

In [40]:
%%time
curr_prices = []
for tick in last_positions['ticker']:
    try:
        price = yf.download(tick, interval='1mo', period='d')['Adj Close'][-1]
    except:
        print(f'No price info for {tick}')
        price = 0
    curr_prices.append(price)
len(curr_prices)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

47

In [41]:
last_positions['price'] = curr_prices
last_positions['current_value'] = (last_positions.price * last_positions.cml_units).round(2)
last_positions['avg_price'] = (last_positions.cml_cost / last_positions.cml_units).round(2)
last_positions = last_positions.sort_values(by='current_value', ascending=False)

last_positions.tail(10)

,ticker,cml_units,cml_cost,gain_loss,cashflow,price,current_value,avg_price
45,ZEN,10.0,811.800,0.000,-811.80,96.879997,968.80,81.18
25,NIO,30.0,846.780,769.680,-77.10,32.150002,964.50,28.23
42,TWTR,22.5,750.525,538.565,-211.96,42.070000,946.57,33.36
39,SYF,20.0,465.900,0.000,-465.90,45.279999,905.60,23.30
33,PTON,20.0,938.100,39.250,-898.85,43.549999,871.00,46.90
8,DKNG,27.5,1080.300,0.000,-1080.30,28.370001,780.18,39.28
4,CSCO,12.0,469.440,0.000,-469.44,56.230000,674.76,39.12
31,PLTR,32.0,846.340,0.000,-846.34,18.980000,607.36,26.45
26,NKLA,50.0,1697.040,-2.070,-1699.11,9.450000,472.50,33.94
29,PAYS,150.0,973.000,0.000,-973.00,1.730000,259.50,6.49


## Final Last Positions

<div class="alert alert-block alert-success">
<b>Finally!</b><br> This last dataframe will give you everything you need to build an awesome dashboard.
</div>

In [42]:
# final_lastpositions = pd.merge(finviz_merged_clean, last_positions, left_on='ticker', right_on='ticker', how='outer')
final_lastpositions = pd.merge(screener_all_clean, last_positions, left_on='ticker', right_on='ticker', how='outer')
final_lastpositions['current_value'] = final_lastpositions.price*final_lastpositions.cml_units
final_lastpositions['avg_price'] = final_lastpositions.cml_cost/final_lastpositions.cml_units
final_lastpositions['portf_weight'] = final_lastpositions.current_value/final_lastpositions.current_value.sum()*100
final_lastpositions['unrealizedval'] = final_lastpositions.current_value - final_lastpositions.cml_cost
final_lastpositions['unrealizedpct'] = final_lastpositions.unrealizedval / final_lastpositions.cml_cost*100
final_lastpositions.round(2)

,ticker,company,sector,industry,p/e,perf_week,perf_month,perf_quart,perf_half,perf_year,...,cml_units,cml_cost,gain_loss,cashflow,price,current_value,avg_price,portf_weight,unrealizedval,unrealizedpct
0,AAPL,Apple Inc.,Technology,Consumer Electronics,28.86,3.21%,6.83%,4.89%,28.55%,31.64%,...,36.00,2735.88,0.00,-2735.88,161.84,5826.24,76.00,6.20,3090.36,112.96
1,AMD,"Advanced Micro Devices, Inc.",Technology,Semiconductors,44.59,-6.98%,10.33%,31.01%,77.03%,56.01%,...,24.00,1296.00,0.00,-1296.00,144.01,3456.24,54.00,3.68,2160.24,166.69
2,AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,66.28,-3.27%,0.17%,-2.54%,6.00%,6.37%,...,1.70,4360.11,0.00,-4360.11,3389.79,5762.64,2564.77,6.13,1402.53,32.17
3,BA,The Boeing Company,Industrials,Aerospace & Defense,-,-0.36%,-6.98%,-9.02%,-21.44%,-16.32%,...,7.80,1644.36,0.00,-1644.36,198.49,1548.22,210.82,1.65,-96.14,-5.85
4,CSCO,"Cisco Systems, Inc.",Technology,Communication Equipment,20.90,2.85%,-2.46%,-5.37%,4.28%,27.48%,...,12.00,469.44,0.00,-469.44,56.23,674.76,39.12,0.72,205.32,43.74
5,CVS,CVS Health Corporation,Healthcare,Healthcare Plans,15.86,-0.71%,-5.68%,3.89%,5.26%,25.30%,...,12.00,656.28,0.00,-656.28,90.87,1090.44,54.69,1.16,434.16,66.15
6,CVX,Chevron Corporation,Energy,Oil & Gas Integrated,22.19,-0.09%,1.24%,17.36%,6.18%,27.41%,...,18.00,1607.32,0.00,-1607.32,114.41,2059.38,89.30,2.19,452.06,28.13
7,DIS,The Walt Disney Company,Communication Services,Entertainment,132.21,-1.28%,-14.03%,-19.22%,-17.39%,-4.58%,...,11.04,1180.84,0.00,-1180.84,146.22,1614.27,106.96,1.72,433.43,36.71
8,DKNG,DraftKings Inc.,Consumer Cyclical,Gambling,-,-21.28%,-39.43%,-53.51%,-46.98%,-43.33%,...,27.50,1080.30,0.00,-1080.30,28.37,780.18,39.28,0.83,-300.12,-27.78
9,DOCU,"DocuSign, Inc.",Technology,Software - Application,-,-45.88%,-52.30%,-56.43%,-43.94%,-41.52%,...,12.00,2941.56,0.00,-2941.56,135.09,1621.08,245.13,1.73,-1320.48,-44.89


In [43]:
# final cosmetics
final_lastpositions = final_lastpositions.replace('-', '0')
for c in final_lastpositions.iloc[:,4:30].columns:
    final_lastpositions[c] = pd.to_numeric(final_lastpositions[c].str.replace('%', ''))
final_lastpositions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 46
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ticker         47 non-null     object 
 1   company        47 non-null     object 
 2   sector         47 non-null     object 
 3   industry       47 non-null     object 
 4   p/e            47 non-null     float64
 5   perf_week      47 non-null     float64
 6   perf_month     47 non-null     float64
 7   perf_quart     47 non-null     float64
 8   perf_half      47 non-null     float64
 9   perf_year      47 non-null     float64
 10  perf_ytd       47 non-null     float64
 11  volatility_w   47 non-null     float64
 12  volatility_m   47 non-null     float64
 13  recom          47 non-null     float64
 14  atr            47 non-null     float64
 15  sma20          47 non-null     float64
 16  sma50          47 non-null     float64
 17  sma200         47 non-null     float64
 18  52w_high    

In [45]:
final_lastpositions.round(2).to_csv('../outputs/final_current_positions/final_current_positions_{}.csv'.format(get_now()), index=False)

## Grouped DF's

In [46]:
grouped_sect = final_lastpositions.groupby(['sector']).agg(
    {'ticker': 'count', 'current_value': 'sum', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by='current_value', ascending= False).reset_index().round(2)
grouped_sect['weight'] = round(grouped_sect.current_value/grouped_sect.current_value.sum()*100, 2)
grouped_sect

,sector,ticker,current_value,cml_cost,gain_loss,weight
0,Technology,15,32770.38,21910.18,2488.50,34.87
1,Consumer Cyclical,9,21662.48,13690.55,3730.14,23.05
2,Communication Services,7,14477.89,8811.33,1985.40,15.41
3,Industrials,5,7453.15,7349.45,2122.52,7.93
4,Financial,3,4853.95,2858.59,0.00,5.17
5,Healthcare,3,3962.83,2082.40,695.85,4.22
6,Consumer Defensive,2,3571.10,3355.94,0.00,3.80
7,Energy,1,2059.38,1607.32,0.00,2.19
8,Basic Materials,1,1614.30,931.30,0.00,1.72
9,Real Estate,1,1544.42,1252.65,0.00,1.64


In [31]:
grouped_sect.to_csv('../outputs/grouped/grouped_sect_{}.csv'.format(get_now()), index=False)

In [47]:
grouped_sect_ind = final_lastpositions.groupby(['sector', 'industry']).agg(
    {'ticker': 'count', 'current_value': 'sum', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by='current_value', ascending= False).reset_index().round(2)
grouped_sect_ind['weight'] = round(grouped_sect_ind.current_value/grouped_sect_ind.current_value.sum()*100, 2)
grouped_sect_ind

,sector,industry,ticker,current_value,cml_cost,gain_loss,weight
0,Consumer Cyclical,Auto Manufacturers,3,10023.65,4702.74,3080.04,10.67
1,Technology,Software - Application,6,9680.58,7977.70,32.34,10.30
2,Technology,Semiconductors,3,9526.26,4369.75,0.00,10.14
3,Communication Services,Internet Content & Information,3,8565.58,4949.50,538.57,9.12
4,Consumer Cyclical,Internet Retail,2,7009.54,4969.11,610.85,7.46
5,Technology,Consumer Electronics,1,5826.24,2735.88,0.00,6.20
6,Communication Services,Entertainment,3,4818.81,3407.83,1034.33,5.13
7,Technology,Software - Infrastructure,2,4483.48,2880.00,0.00,4.77
8,Financial,Credit Services,2,3112.76,1785.78,0.00,3.31
9,Consumer Defensive,Tobacco,1,2196.00,2113.60,0.00,2.34


In [33]:
grouped_sect_ind.to_csv('../outputs/grouped/grouped_sect_ind_{}.csv'.format(get_now()), index=False)